In [2]:
import pandas as pd
import numpy as np

off_train = pd.read_csv('C://Users/Administrator/Downloads/ccf_offline_stage1_train.csv')
#print(off_train.head())

off_test = pd.read_csv('C://Users/Administrator/Downloads/ccf_offline_stage1_test_revised.csv')
#print(off_test.head())

on_train = pd.read_csv('C://Users/Administrator/Downloads/ccf_online_stage1_train (1).csv')
#print(on_train.head())

dataset3= off_test
#改
dataset2= off_train[((off_train.Date>='20160515')&(off_train.Date<='20160630')) | ((off_train.Date=='null') & (off_train.Date_received>='20160515') & (off_train.Date_received<='20160630'))]
dataset1= off_train[(off_train.Date_received!='null')& (off_train.Date!='null')]
dataset0= off_train[(off_train.Date_received!='null')& (off_train.Date=='null')]
dataset= off_train[(off_train.Date_received=='null')& (off_train.Date!='null')]
#print(dataset1,dataset0,dataset)
dataset1['label']=1
dataset0['label']=-1
dataset['label']=0
dataset1=pd.concat([dataset1,dataset0],axis=0)
print(dataset1,dataset)                                                                                                                                                                


t=dataset3[['User_id']]
t['this_month_user_receive_all_coupon_count']=1
t=t.groupby('User_id').agg('sum').reset_index()

t1 = dataset3[['User_id','Coupon_id']]

t1['this_month_user_receive_same_coupon_count'] = 1

t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()

t2= dataset3[['User_id','Coupon_id','Date_received']]
t2.Date_received= t2.Date_received.astype('str')
t2=t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
print(t2.head())
t2['received_number']=t2.Date_received.apply(lambda s:len(s.split(':')))
#print(t2)
t2=t2[t2['received_number']>1]
t2['max_date_received']=t2.Date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
t2['min_date_received']=t2.Date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
t2=t2[['User_id','Coupon_id','max_date_received','min_date_received']]

t3 = dataset3[['User_id','Coupon_id','Date_received']]

t3 = pd.merge(t3,t2,on=['User_id','Coupon_id'],how='left')

t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.Date_received

t3['this_month_user_receive_same_coupon_firstone'] = t3.Date_received - t3.min_date_received

def is_firstlastone(x):

    if x==0:

        return 1

    elif x>0:

        return 0

    else:

        return -1 #those only receive once

        

t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)

t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)

t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]

t4 = dataset3[['User_id','Date_received']]

t4['this_day_user_receive_all_coupon_count'] = 1

t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()

t5 = dataset3[['User_id','Coupon_id','Date_received']]

t5['this_day_user_receive_same_coupon_count'] = 1

t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()

from datetime import datetime
t6 = dataset3[['User_id','Coupon_id','Date_received']]
t6.Date_received = t6.Date_received.astype('str')
t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'Date_received':'dates'},inplace=True)


other_feature3= pd.merge(t1,t,on='User_id')
other_feature3= pd.merge(other_feature3,t3,on=['User_id','Coupon_id'],how='left')
other_feature3=pd.merge(other_feature3,t4,on=['User_id','Date_received'])
other_feature3= pd.merge(other_feature3,t5,on=['User_id','Coupon_id','Date_received'])
print(other_feature3.head(5))
other_feature3.to_csv('data/1other_feature3.csv',index=None)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


         User_id  Merchant_id Coupon_id Discount_rate Distance Date_received  \
6        1439408         2632      8591          20:1        0      20160516   
33       1113008         1361     11166          20:1        0      20160515   
38       2881376         8390      7531          20:5        0      20160321   
69        114747         6901      2366          30:5        0      20160523   
75        114747         5341       111          30:5        0      20160127   
76        114747         5341       111          30:5        0      20160207   
77        114747         5341      7751         50:10        0      20160127   
84        114747         2454      8088          20:5        0      20160324   
100       114747         3381      9776          10:5        0      20160129   
143       205174         1169      4627         50:20        0      20160206   
192      1380272         2099     12034        100:10        1      20160201   
193      1380272         5341      7751 

D:\anaconda\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = v

   User_id  Coupon_id Date_received
0      209        825      20160721
1      209       7557      20160721
2      215       5488      20160703
3      316       3992      20160721
4      417      12465      20160712


D:\anaconda\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   User_id  Coupon_id  this_month_user_receive_same_coupon_count  \
0      209        825                                          1   
1      209       7557                                          1   
2      215       5488                                          1   
3      316       3992                                          1   
4      417      12465                                          1   

   this_month_user_receive_all_coupon_count  Date_received  \
0                                         2       20160721   
1                                         2       20160721   
2                                         1       20160703   
3                                         1       20160721   
4                                         1       20160712   

   this_month_user_receive_same_coupon_lastone  \
0                                           -1   
1                                           -1   
2                                           -1   
3                 

In [3]:
t= dataset1[['User_id']]
t['this_month_receive_all_coupon_id']=1
t= t.groupby('User_id').agg('sum').reset_index()
#print(t)

t1 = dataset1[['User_id','Coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()
#print(t1)

t2 = dataset1[['User_id','Coupon_id','Date_received']]

#t2= t2[t2.Date_received!='null']

t2.Date_received = t2.Date_received.astype('str')

t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()

t2['receive_number'] = t2.Date_received.apply(lambda s:len(s.split(':')))

t2 = t2[t2.receive_number>1]

t2['max_date_received'] = t2.Date_received.apply(lambda s:max([int(d) for d in s.split(':') ]))

t2['min_date_received'] = t2.Date_received.apply(lambda s:min([int(d) for d in s.split(':') ]))

t2 = t2[['User_id','Coupon_id','max_date_received','min_date_received']]

t3=dataset1[['User_id','Coupon_id','Date_received']]
t3= pd.merge(t3,t2,on=['Coupon_id','User_id'],how='left')
#print(t3)

t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.Date_received.astype('int')

t3['this_month_user_receive_same_coupon_firstone'] = t3.Date_received.astype('int') - t3.min_date_received

def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1

t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)

t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)

t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]


t4 = dataset1[['User_id','Date_received']]

t4['this_day_user_receive_all_coupon_count'] = 1

t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()



t5 = dataset1[['User_id','Coupon_id','Date_received']]

t5['this_day_user_receive_same_coupon_count'] = 1

t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()



t6 = dataset1[['User_id','Coupon_id','Date_received']]

t6.Date_received = t6.Date_received.astype('str')

t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()

t6.rename(columns={'Date_received':'dates'},inplace=True)


other_feature2 = pd.merge(t1,t,on='User_id')

other_feature2 = pd.merge(other_feature2,t3,on=['User_id','Coupon_id'])

other_feature2 = pd.merge(other_feature2,t4,on=['User_id','Date_received'])

other_feature2 = pd.merge(other_feature2,t5,on=['User_id','Coupon_id','Date_received'])

#other_feature2 = pd.merge(other_feature2,t7,on=['User_id','Coupon_id','Date_received'])

print(other_feature2.head(5))

other_feature2.to_csv('data/1other_feature2.csv',index=None)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\anaconda\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  

   User_id Coupon_id  this_month_user_receive_same_coupon_count  \
0        4      2902                                          1   
1        4      8735                                          1   
2       35     11951                                          2   
3       35      9776                                          1   
4       35     11951                                          2   

   this_month_receive_all_coupon_id Date_received  \
0                                 2      20160607   
1                                 2      20160214   
2                                 4      20160129   
3                                 4      20160129   
4                                 4      20160130   

   this_month_user_receive_same_coupon_lastone  \
0                                           -1   
1                                           -1   
2                                            0   
3                                           -1   
4                           

In [4]:
#coupon_feature

def calc_discount_rate(s):
    s=str(s)
    s= s.split(':')
    if len(s)==1:
        return float(s[0])
    else:
        return 1.0-float(s[1])/float(s[0])
    
def get_discount_man(s):
    s= str(s)
    s= s.split(':')
    if len(s)==1:
        return 100
    else:
        return int(s[0])

def get_discount_jian(s):
    s=str(s)
    s= s.split(':')
    if len(s)==1:
        return 100*float(s[0])
    else:
        return int(s[1])

def is_man_jian(s):
    s= str(s)
    s= s.split(':')
    if len(s)==1:
        return 0
    else:
        return 1
    
dataset3['discount_man']= dataset3.Discount_rate.apply(get_discount_man)
dataset3['discount_jian'] = dataset3.Discount_rate.apply(get_discount_jian)

dataset3['is_man_jian'] = dataset3.Discount_rate.apply(is_man_jian)

dataset3['discount_rate'] = dataset3.Discount_rate.apply(calc_discount_rate)
#dataset3.drop('coupon_count_y',axis=1,inplace=True)
#dataset3.drop('Discount_rate',axis=1,inplace=True)
dataset30=dataset3.drop('User_id',axis=1)
print(dataset30.head())
dataset30.to_csv('data/1coupon3_feature.csv',index=None)

   Merchant_id  Coupon_id Discount_rate Distance  Date_received  discount_man  \
0          450       9983          30:5        1       20160712            30   
1         1300       3429          30:5     null       20160706            30   
2         7113       6928        200:20        5       20160727           200   
3         7113       1808        100:10        5       20160727           100   
4         7605       6500          30:1        2       20160708            30   

   discount_jian  is_man_jian  discount_rate  
0            5.0            1       0.833333  
1            5.0            1       0.833333  
2           20.0            1       0.900000  
3           10.0            1       0.900000  
4            1.0            1       0.966667  


In [5]:
dataset1['discount_man']= dataset1.Discount_rate.apply(get_discount_man)
dataset1['discount_jian'] = dataset1.Discount_rate.apply(get_discount_jian)

dataset1['is_man_jian'] = dataset1.Discount_rate.apply(is_man_jian)

dataset1['discount_rate'] = dataset1.Discount_rate.apply(calc_discount_rate)
#dataset3.drop('coupon_count_y',axis=1,inplace=True)
#dataset3.drop('Discount_rate',axis=1,inplace=True)
dataset10=dataset1.drop('User_id',axis=1)
print(dataset10.head())
dataset10.to_csv('data/1coupon2_feature.csv',index=None)

    Merchant_id Coupon_id Discount_rate Distance Date_received      Date  \
6          2632      8591          20:1        0      20160516  20160613   
33         1361     11166          20:1        0      20160515  20160521   
38         8390      7531          20:5        0      20160321  20160329   
69         6901      2366          30:5        0      20160523  20160605   
75         5341       111          30:5        0      20160127  20160221   

    label  discount_man  discount_jian  is_man_jian  discount_rate  
6       1            20            1.0            1       0.950000  
33      1            20            1.0            1       0.950000  
38      1            20            5.0            1       0.750000  
69      1            30            5.0            1       0.833333  
75      1            30            5.0            1       0.833333  


In [6]:
#merchant related feature
#for dataset3
merchant3= dataset2[['Merchant_id','Coupon_id','Distance','Date_received','Date']]

t=merchant3[['Merchant_id']]
t.drop_duplicates(inplace=True)

t1= merchant3[merchant3.Date!='null'][['Merchant_id']]
t1['total_sales']=1
t1=t1.groupby('Merchant_id').agg('sum').reset_index()
#print(t1)

t2= merchant3[(merchant3.Coupon_id !='null') & (merchant3.Date !='null')][['Merchant_id']]
t2['sales_use_coupon']=1
t2= t2.groupby('Merchant_id').agg('sum').reset_index()

t3= merchant3[merchant3.Coupon_id !='null'][['Merchant_id']]
t3['total_coupon']=1
t3= t3.groupby('Merchant_id').agg('sum').reset_index()

merchant3_feature= pd.merge(t,t1,on='Merchant_id',how='left')
merchant3_feature= pd.merge(merchant3_feature,t2,on='Merchant_id',how='left')
merchant3_feature= pd.merge(merchant3_feature,t3,on='Merchant_id',how='left')

merchant3_feature.sales_use_coupon= merchant3_feature.sales_use_coupon.replace(np.nan,0)
merchant3_feature['merchant_coupon_transfer_rate']= merchant3_feature.sales_use_coupon.astype('float')/merchant3_feature.total_coupon
merchant3_feature['coupon_rate']= merchant3_feature.sales_use_coupon.astype('float')/merchant3_feature.total_sales
merchant3_feature.total_coupon=merchant3_feature.total_coupon.replace(np.nan,0)
merchant3_feature.merchant_coupon_transfer_rate= merchant3_feature.merchant_coupon_transfer_rate.replace(np.nan,0)
merchant3_feature.coupon_rate= merchant3_feature.coupon_rate.replace(np.nan,0)
print(merchant3_feature)

merchant3_feature.to_csv('data/1merchant3_feature.csv',index=None)


D:\anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


      Merchant_id  total_sales  sales_use_coupon  total_coupon  \
0            4663        141.0              31.0       10855.0   
1            2632          3.0               1.0           8.0   
2             450       3793.0             791.0       20472.0   
3            6459         15.0               0.0          16.0   
4            6901       3894.0             604.0        9422.0   
5            1579         27.0               1.0          12.0   
6            3051          2.0               0.0           0.0   
7            3621       2036.0             852.0       17548.0   
8            2277        257.0              41.0        1623.0   
9            1361         38.0              11.0          16.0   
10           6485       5640.0             705.0        3255.0   
11           4874        170.0               4.0          93.0   
12           1539         14.0               0.0           0.0   
13           4666        213.0               0.0         773.0   
14        

In [7]:
#user related
from datetime import  datetime
def get_user_date_datereceived_gap(s):
    s= s.split(':')
    return (datetime(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8]))-datetime(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days

user3= dataset2[['User_id','Merchant_id','Coupon_id','Discount_rate','Distance','Date_received','Date']]
t= user3[['User_id']]
t.drop_duplicates(inplace=True)

t1= user3[user3.Date!= 'null'][['User_id','Merchant_id']]
t1.drop_duplicates(inplace=True)
t1.Merchant_id =1
t1= t1.groupby('User_id').agg('sum').reset_index()
t1.rename(columns={'Merchant_id':'count_merchant'},inplace=True)

t7 = user3[(user3.Date!='null')&(user3.Coupon_id!='null')][['User_id']]

t7['buy_use_coupon'] = 1

t7 = t7.groupby('User_id').agg('sum').reset_index()

t8 = user3[user3.Date!='null'][['User_id']]

t8['buy_total'] = 1

t8 = t8.groupby('User_id').agg('sum').reset_index()

t9 = user3[user3.Coupon_id!='null'][['User_id']]

t9['coupon_received'] = 1

t9 = t9.groupby('User_id').agg('sum').reset_index()

user3_feature = pd.merge(t,t1,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t7,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t8,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t9,on='User_id',how='left')

user3_feature.count_merchant= user3_feature.count_merchant.replace(np.nan,0)
user3_feature.buy_use_coupon= user3_feature.buy_use_coupon.replace(np.nan,0)
user3_feature['buy_use_coupon_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.buy_total.astype('float')
user3_feature['user_coupon_transfer_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.coupon_received.astype('float')
user3_feature.buy_total = user3_feature.buy_total.replace(np.nan,0)
user3_feature.coupon_received = user3_feature.coupon_received.replace(np.nan,0)
user3_feature.buy_use_coupon_rate= user3_feature.buy_use_coupon_rate.replace(np.nan,0)
user3_feature.user_coupon_transfer_rate= user3_feature.user_coupon_transfer_rate.replace(np.nan,0)
user3_feature.to_csv('data/1user3_feature.csv',index=None)
print(user3_feature.head())


D:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


   User_id  count_merchant  buy_use_coupon  buy_total  coupon_received  \
0  1439408             1.0             1.0        2.0              3.0   
1  2029232             1.0             0.0        2.0              2.0   
2  2747744             0.0             0.0        0.0              1.0   
3   196342             1.0             0.0        1.0              1.0   
4   253750             0.0             0.0        0.0              1.0   

   buy_use_coupon_rate  user_coupon_transfer_rate  
0                  0.5                   0.333333  
1                  0.0                   0.000000  
2                  0.0                   0.000000  
3                  0.0                   0.000000  
4                  0.0                   0.000000  


In [8]:
#user merchant
#times user buy merchant before

all_user_merchant= dataset2[['User_id','Merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)

t= dataset2[['User_id','Merchant_id','Date']]
t= t[t.Date!= 'null'][['User_id','Merchant_id']]
t['user_merchant_buy_total']=1
t= t.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)

t1 = dataset2[['User_id','Merchant_id','Coupon_id']]

t1 = t1[t1.Coupon_id!='null'][['User_id','Merchant_id']]

t1['user_merchant_received'] = 1

t1 = t1.groupby(['User_id','Merchant_id']).agg('sum').reset_index()

t1.drop_duplicates(inplace=True)


t2 = dataset2[['User_id','Merchant_id','Date','Date_received']]

t2 = t2[(t2.Date!='null')&(t2.Date_received!='null')][['User_id','Merchant_id']]

t2['user_merchant_buy_use_coupon'] = 1

t2 = t2.groupby(['User_id','Merchant_id']).agg('sum').reset_index()

t2.drop_duplicates(inplace=True)


t3 = dataset2[['User_id','Merchant_id']]

t3['user_merchant_any'] = 1

t3 = t3.groupby(['User_id','Merchant_id']).agg('sum').reset_index()

t3.drop_duplicates(inplace=True)


t4 = dataset2[['User_id','Merchant_id','Date','Coupon_id']]

t4 = t4[(t4.Date!='null')&(t4.Coupon_id=='null')][['User_id','Merchant_id']]

t4['user_merchant_buy_common'] = 1

t4 = t4.groupby(['User_id','Merchant_id']).agg('sum').reset_index()

t4.drop_duplicates(inplace=True)


user_merchant3 = pd.merge(all_user_merchant,t,on=['User_id','Merchant_id'],how='left')

user_merchant3 = pd.merge(user_merchant3,t1,on=['User_id','Merchant_id'],how='left')

user_merchant3 = pd.merge(user_merchant3,t2,on=['User_id','Merchant_id'],how='left')

user_merchant3 = pd.merge(user_merchant3,t3,on=['User_id','Merchant_id'],how='left')

user_merchant3 = pd.merge(user_merchant3,t4,on=['User_id','Merchant_id'],how='left')

user_merchant3.user_merchant_buy_use_coupon = user_merchant3.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant3.user_merchant_buy_common = user_merchant3.user_merchant_buy_common.replace(np.nan,0)

user_merchant3['user_merchant_coupon_transfer_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_received.astype('float')

user_merchant3['user_merchant_coupon_buy_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')

user_merchant3['user_merchant_rate'] = user_merchant3.user_merchant_buy_total.astype('float') / user_merchant3.user_merchant_any.astype('float')

user_merchant3['user_merchant_common_buy_rate'] = user_merchant3.user_merchant_buy_common.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')
user_merchant3= user_merchant3.fillna(value=0)
user_merchant3.to_csv('data/1user_merchant3.csv',index=None)
print(user_merchant3.head())

D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   User_id  Merchant_id  user_merchant_buy_total  user_merchant_received  \
0  1439408         4663                      0.0                     1.0   
1  1439408         2632                      2.0                     2.0   
2  2029232          450                      0.0                     1.0   
3  2029232         6459                      2.0                     1.0   
4  2747744         6901                      0.0                     1.0   

   user_merchant_buy_use_coupon  user_merchant_any  user_merchant_buy_common  \
0                           0.0                  1                       0.0   
1                           1.0                  3                       1.0   
2                           0.0                  1                       0.0   
3                           0.0                  3                       2.0   
4                           0.0                  1                       0.0   

   user_merchant_coupon_transfer_rate  user_merchant_coupon_bu

In [10]:
def get_label(s):
    s=s.split(':')
    if s[0]=='null':
        return 0
    elif (datetime(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8]))-datetime(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days<=15:
        return 1
    else:
        return -1

'''   
off_train= pd.read_csv('ccf_offline_stage1_train.csv')
dataset10= pd.read_csv('1coupon2_feature.csv')
user3_feature=pd.read_csv('1user3_feature.csv')
user_merchant3= pd.read_csv('1user_merchant3.csv')
other_feature2= pd.read_csv('1other_feature2.csv')
'''
#改
#of_train= off_train[((off_train.Date>='20160515')&(off_train.Date<='20160630')) | ((off_train.Date=='null') & (off_train.Date_received>='20160515') & (off_train.Date_received<='20160630'))]

dataset4= pd.merge(off_train,dataset10,on=['Merchant_id','Coupon_id'],how='left')
'''
dataset4= pd.merge(dataset4,merchant3_feature,on='Merchant_id',how='left')
dataset4= pd.merge(dataset4,user3_feature,on='User_id',how='left')
dataset4= pd.merge(dataset4,user_merchant3,on=['User_id','Merchant_id'],how='left')
dataset4= pd.merge(dataset4,other_feature2,on=['User_id','Coupon_id','Date_received'],how='left')

dataset4.drop_duplicates(inplace=True)
'''
print(dataset4.head())


MemoryError: 